In [103]:
import pandas as pd
import sqlite3
import seaborn as sns
import matplotlib.pyplot as plt
%matplotlib inline
conn = sqlite3.connect('data/im.db')
cur = conn.cursor()

df = pd.read_csv('data/bom.movie_gross.csv.gz', index_col=0)
df.head(10)

,studio,domestic_gross,foreign_gross,year
title,,,,
Toy Story 3,BV,415000000.0,652000000,2010
Alice in Wonderland (2010),BV,334200000.0,691300000,2010
Harry Potter and the Deathly Hallows Part 1,WB,296000000.0,664300000,2010
Inception,WB,292600000.0,535700000,2010
Shrek Forever After,P/DW,238700000.0,513900000,2010
The Twilight Saga: Eclipse,Sum.,300500000.0,398000000,2010
Iron Man 2,Par.,312400000.0,311500000,2010
Tangled,BV,200800000.0,391000000,2010
Despicable Me,Uni.,251500000.0,291600000,2010


In [19]:
pd.read_sql("""
SELECT * 
  FROM movie_ratings
  where averagerating >= 8 AND numvotes >= 5000
""", conn)







,movie_id,averagerating,numvotes
0,tt1663202,8.0,621193
1,tt8108198,8.5,43409
2,tt1201607,8.1,691835
3,tt1255953,8.3,124156
4,tt1649431,8.2,5707
...,...,...,...
207,tt1424432,8.6,55318
208,tt1821700,8.1,28950
209,tt4154756,8.5,670926
210,tt3569782,8.3,8484


In [18]:
pd.read_sql("""
SELECT movie_id , primary_title , original_title , genres
    FROM movie_basics
""", conn)

,movie_id,primary_title,original_title,genres
0,tt0063540,Sunghursh,Sunghursh,"Action,Crime,Drama"
1,tt0066787,One Day Before the Rainy Season,Ashad Ka Ek Din,"Biography,Drama"
2,tt0069049,The Other Side of the Wind,The Other Side of the Wind,Drama
3,tt0069204,Sabse Bada Sukh,Sabse Bada Sukh,"Comedy,Drama"
4,tt0100275,The Wandering Soap Opera,La Telenovela Errante,"Comedy,Drama,Fantasy"
...,...,...,...,...
146139,tt9916538,Kuambil Lagi Hatiku,Kuambil Lagi Hatiku,Drama
146140,tt9916622,Rodolpho Teóphilo - O Legado de um Pioneiro,Rodolpho Teóphilo - O Legado de um Pioneiro,Documentary
146141,tt9916706,Dankyavar Danka,Dankyavar Danka,Comedy
146142,tt9916730,6 Gunn,6 Gunn,None


In [52]:
q = """
SELECT person_id , primary_name as Director , COUNT(directors.movie_id) as "Movies Made"  
    FROM persons
    JOIN directors 
      USING (person_id)
    JOIN movie_basics
      USING (movie_id)
    GROUP BY person_id
    ORDER BY "Movies Made" DESC
    
;
"""
pd.read_sql(q, conn).head(20)

,person_id,Director,Movies Made
0,nm6935209,Tony Newton,238
1,nm2563700,Jason Impey,190
2,nm1546474,Shane Ryan,185
3,nm3877467,Ruben Rodriguez,180
4,nm3848412,Sam Mason-Bell,144
5,nm6078996,Martin Sonntag,135
6,nm4341114,Dustin Ferguson,121
7,nm4921301,Gav Chuckie Steel,119
8,nm3079464,Corey Norman,114
9,nm4542885,Evan Marlowe,101


In [80]:
avg_rating = """
SELECT person_id , primary_name as Director , COUNT(directors.movie_id) as "Movies Made",  sum(movie_ratings.averagerating) / COUNT(movie_ratings.averagerating) as "Average Rating"
, movie_ratings.numvotes as numvotes
    FROM persons
    JOIN directors 
      USING (person_id)
    JOIN movie_basics
      USING (movie_id)
    JOIN movie_ratings
      USING (movie_id)
    GROUP BY person_id
    HAVING "Movies Made" >= 5 AND "Average Rating" >= 8 AND numvotes >= 1000
    ORDER BY "Average Rating" DESC
    
;
"""
pd.read_sql(avg_rating, conn)

,person_id,Director,Movies Made,Average Rating,numvotes
0,nm3586222,Thiagarajan Kumararaja,5,8.760000,2769
1,nm0000233,Quentin Tarantino,5,8.700000,1211405
2,nm0634240,Christopher Nolan,8,8.437500,1299334
3,nm0062614,Harry Baweja,5,8.360000,7451
4,nm0881279,Lee Unkrich,8,8.350000,682218
5,nm4274575,Vetrimaaran,5,8.340000,3839
6,nm9751348,Rahi Anil Barve,5,8.300000,14128
7,nm4205110,Adesh Prasad,5,8.300000,14128
8,nm0081449,Dragan Bjelogrlic,6,8.300000,10502
9,nm2609716,Tauquir Ahmed,5,8.260000,2973


In [111]:
Quentin = """
SELECT directors.person_id , persons.primary_name,  movie_basics.primary_title ,  movie_ratings.averagerating 
FROM principals 
JOIN persons
      USING (person_id)
JOIN movie_basics
      USING (movie_id)
JOIN directors 
      USING (person_id)
      JOIN movie_ratings
      USING (movie_id)
WHERE person_id = 'nm0000233' 
GROUP BY movie_basics.primary_title

"""
pd.read_sql(Quentin, conn)

,person_id,primary_name,primary_title,averagerating
0,nm0000233,Quentin Tarantino,Bar25,7.1
1,nm0000233,Quentin Tarantino,Corman's World: Exploits of a Hollywood Rebel,7.6
2,nm0000233,Quentin Tarantino,Django Unchained,8.4
3,nm0000233,Quentin Tarantino,Hollywood Don't Surf!,8.0
4,nm0000233,Quentin Tarantino,John Travolta: Rückkehr nach Hollywood,5.9
5,nm0000233,Quentin Tarantino,Kill Bill: The Whole Bloody Affair,8.8
6,nm0000233,Quentin Tarantino,Once Upon a Time ... in Hollywood,9.7
7,nm0000233,Quentin Tarantino,The Hateful Eight,7.8


In [112]:
df2 = pd.read_sql(avg_rating,conn)
df2.isna().sum()

person_id         0
Director          0
Movies Made       0
Average Rating    0
numvotes          0
dtype: int64

In [115]:
df=pd.read_sql(Quentin, conn)
df.pivot(index='',columns='primary_title',values='averagerating').plot(kind='bar',figsize=(15,8))
plt.xlabel("primary_title")
plt.ylabel('average rating')
plt.title("Quentin's movies")
plt.show()

ValueError: The name primary_title occurs multiple times, use a level number